In [1]:
from src.pipeline_4_agent import SequentialAgentPipeline
from src.retrieval_then_llm import SmartClassifier

ticket = """
'П. Дэжидмаа': "Get started"
'customer service': "Сайн байна уу? 👋 Toki апп-тай холбоотой бүх төрлийн асуултуудаа бичээрэй 😊"
'customer service': "Та төлбөрийн гүйлгээний хуулга эсвэл үйлчилгээтэй холбоотой алдааны зураг илгээсэн үү 😊"
'П. Дэжидмаа': "uul ni toltson baihad yagaad ingeed garj ireed baigaa ym boloo"
'П. Дэжидмаа': "uuchlarai anduurtsan bn"
'customer service': "Сайн байна уу? Би Уянга байна 🙋‍♀️ Танд өөр асууж тодруулах зүйл гарвал бичээрэй. Бид танд туслахдаа үргэлж баяртай байх болно 😊"
'П. Дэжидмаа': "uuchlarai anduurtsan bn"
'customer service': "Бид танд туслахдаа үргэлж баяртай байх болно. Toki-г сонгосонд баярлалаа😊️💛️"
"""

In [2]:
pipeline = SequentialAgentPipeline("data/cleaned.csv")
final_categorization = await pipeline.run(ticket)

Loading and preparing taxonomy from data/cleaned.csv...
Toki>Бүртгэл, тохиргоо>Бүртгүүлэхтэй холбоотой>L2-Хэрэглэгчээс шалтгаалсан


In [2]:
classifier = SmartClassifier(csv_path="data/cleaned.csv")
result = await classifier.classify(ticket)

Initializing Smart Classifier (Domain→Cat1→Cat2→Cat3)...
Loading existing FAISS index from taxonomy.index...


In [3]:
print(result["path_mn"])

Toki > Үйлчилгээ > Toki Power > Төлөлт,  гүйлгээтэй холбоотой
